### Funciona bem para maquinas T14 g3, g4 e g5

In [192]:
def registrar_maquinas_nao_informadas(serial_number, specs_string, arquivo_log):
    """
    Registra no arquivo de log o número de série e especificações caso haja campos 'Não informado'.
    """
    try:
        with open(arquivo_log, "a", encoding="utf-8") as log_file:
            log_file.write(f"Serial Number: {serial_number}\n")
            log_file.write(f"{specs_string}\n")
            log_file.write("------------\n")
        print(f"Máquina registrada com sucesso no log: {serial_number}")
    except Exception as e:
        print(f"Erro ao salvar no arquivo de log: {e}")


In [194]:
def extrair_especificacoes(serial_number, specs_string):
    """
    Extrai os valores que estão na linha seguinte a cada categoria, considerando traduções,
    e retorna um dicionário com os valores ou 'Não informado' caso não seja encontrado.
    """
    categorias = {
        "Processador": "Processor",
        "Memória": "Memory",
        "Sistema Operacional": "Operating System",
        "Hard Drive": "Hard Drive",
        "Rede cabeada": "Wired Network",
        "Rede sem fio": "Wireless Network",
        "Portas": "Ports",
        "Câmera": "Camera",
        "Gráficos": "Graphics",
        "Monitores": "Monitor",
        "Form Factor": "Form Factor",
        "Garantia inclusa": "Included Warranty",
        "Fim de serviço": "End of Service"
    }

    valores_extraidos = {categoria: "Não informado" for categoria in categorias}  # Inicia todas como "Não informado"
    linhas = specs_string.split("\n")  # Divide o texto por linha
    
    for i, linha in enumerate(linhas):
        linha_limpa = linha.strip()

        # Se a linha for uma das categorias em português ou inglês
        if linha_limpa in categorias.keys() or linha_limpa in categorias.values():
            if i + 1 < len(linhas):  # Verifica se há uma linha seguinte
                # Determina a chave em português da categoria
                categoria_chave = linha_limpa if linha_limpa in categorias.keys() else \
                    next(k for k, v in categorias.items() if v == linha_limpa)
                valores_extraidos[categoria_chave] = linhas[i + 1].strip()  # Extrai a próxima linha como valor


    arquivo_log = "minha_lista_de_maquinas.txt" 
    
        # Chamar registrar_maquinas_nao_informadas se houver campos "Não informado"
    if any(valor == "Não informado" for valor in valores_extraidos.values()):
        registrar_maquinas_nao_informadas(serial_number, specs_string, arquivo_log)
    
    return valores_extraidos


In [196]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


# Caminho para o ChromeDriver (não o navegador Chrome em si)
service = Service(r"chromedriver.exe")  # ou onde estiver seu chromedriver

# Inicializa o navegador
driver = webdriver.Chrome(service=service)


# Carregar o arquivo Excel
file_path = "test.xlsx"
workbook = load_workbook(file_path)
sheet = workbook.active




In [197]:
# Configurar o tempo de espera explícita
wait = WebDriverWait(driver, 20)

# Percorrer as células da coluna A
for row in range(2, sheet.max_row + 1):  # Começando da linha 2 para ignorar o cabeçalho
    serial_number = sheet[f'A{row}'].value

    # Acessar o site da Lenovo Warranty Lookup
    driver.get("https://pcsupport.lenovo.com/us/en/products/laptops-and-netbooks/thinkpad-x-series-laptops/thinkpad-x380-yoga/20lj/20lja029br/mp1en401/warranty/?linkTrack=Caps:Body_SearchProduct&searchType=6&keyWordSearch=mp1en401")
    time.sleep(5) 

    # Esperar o campo de input estar presente
    try:


        # Aguardar o campo correto para inserir o número de série
        input_field = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/section[2]/div[1]/div[2]/div[2]/div[2]/span[1]/input[2]')))
        
        # Clicar no campo de entrada para habilitar a edição                
        input_field.click()
        # Tentar limpar o campo usando uma combinação de teclas
        input_field.send_keys(Keys.CONTROL + "a")  # Selecionar todo o texto no campo
        input_field.send_keys(Keys.DELETE)  # Deletar o texto selecionado
        print (serial_number)
        input_field.clear()
        input_field.send_keys(serial_number)
        input_field.send_keys(Keys.RETURN)

        time.sleep(5)  # Esperar a página carregar os resultados

        # Extrair o texto do campo de status da garantia
        status_xpath = '/html/body/div[2]/section[2]/div[1]/div[2]/div[2]/div[1]/h4'  # Ajuste este XPath conforme o conteúdo correto
        status_element = wait.until(EC.presence_of_element_located((By.XPATH, status_xpath)))
        status_text = status_element.text
        sheet[f'B{row}'] = status_text

        # Extrair o conteúdo do campo relacionado à garantia
        warranty_xpath = '//*[@id="app-psp-warranty"]/div[2]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/span[2]'  # Ajuste conforme o conteúdo correto
        warranty_element = wait.until(EC.presence_of_element_located((By.XPATH, warranty_xpath)))
        warranty_text = warranty_element.text
        sheet[f'C{row}'] = warranty_text
        
                
        # Encontra o elemento "Product home" pelo XPath e clica nele
        elemento = driver.find_element(By.XPATH, "/html/body/div[2]/section[2]/div[1]/div[3]/ul/li[1]/a/span[2]")
        elemento.click()
        
        time.sleep(5)
            
        # Encontra o elemento "Product View" pelo XPath e clica nele
        elemento = driver.find_element(By.XPATH, "//*[@id='app-psp-home']/div/div[2]/div[2]/section/div/div/div[4]/div[3]/div[2]/div/span")
        elemento.click()
        
        
        # Encontra o elemento pelo XPath e clica nele
        
        produtoEspec = '//*[@id="app-psp-home"]/div/div[2]/div[2]/section/div/div/div[4]/div[3]/div[2]/div/div'  # Ajuste conforme o conteúdo correto
        produtoEspecWait = wait.until(EC.presence_of_element_located((By.XPATH, produtoEspec)))
        produtoEspecText = produtoEspecWait.text
     


        listaEspecificacoes = extrair_especificacoes(serial_number, produtoEspecText)


        # Verifica se existe algum campo "Não informado"
        if all(valor != "Não informado" for valor in listaEspecificacoes.values()):  # Só procede se todos os valores forem válidos
            # Preenche a planilha somente se todos os campos forem válidos
            for col_offset, valor in enumerate(listaEspecificacoes.values()):
                print(valor)  # Exibe o valor para depuração
                sheet.cell(row=row, column=4 + col_offset, value=valor)  # Começa na coluna D
        else:
            print(f"Valores 'Não informado' encontrados para o serial number {serial_number}. Linha não será populada.")







        """
        # Preenche a planilha somente com os valores extraídos
        for col_offset, valor in enumerate(listaEspecificacoes.values()):  # Usamos apenas os valores, ignorando as chaves
            print(valor)  # Exibe o valor para depuração
            sheet.cell(row=row, column=4 + col_offset, value=valor)  # Começa na coluna D

"""












        """
        listaEspecificacoes = extrair_especificacoes(produtoEspecText)
        
        #print(f"Salvando linha {row}: {listaEspecificacoes}")

        for col_offset, valor in enumerate(listaEspecificacoes.items()):
            print(valor)
            sheet.cell(row=row, column=4 + col_offset, value=valor)  # Começa na coluna D
        
       """ 

    except Exception as e:
        print(f"Erro na linha {row}: {e}")
        

    # Salvar o arquivo Excel após cada iteração
    workbook.save(file_path)

# Fechar o navegador e encerrar a sessão
#driver.quit()


PF46ZJD4
1x AMD Ryzen™ 7 PRO 6850U Processor(Ryzen™ 7 PRO 6850U)
1x 32GBLPDDR5-6400
Windows 11 Pro 64 (preinstallato con downgrade a Windows 10 Pro 64)(BR:Brazilian Portuguese)
1x 256 GB SSD PCIe
1x
1x Bluetooth® 5.1 or above; Qualcomm Wi-Fi 6E NFA725A 2x2 AX
1x Headphone & Microphone combo Jack; 2 USB 3.2 Gen 2 (Type A); 2 x USB 4.0 Type-C with PD & DP (Thunderbolt); 1 x RJ45; 1 x HDMI 2.0b; Kensington lock
1x 1080P FHD IR/RGB Hybrid with Dual Array Microphone and Privacy Shutter
1x AMD Radeon™ 680M
14" WUXGA
Ultraslim Notebook
3 Year On-site
2028-12-02
PF46ZJD7
1x AMD Ryzen™ 7 PRO 6850U Processor(Ryzen™ 7 PRO 6850U)
1x 32GBLPDDR5-6400
Windows 11 Pro 64 (preinstallato con downgrade a Windows 10 Pro 64)(BR:Brazilian Portuguese)
1x 256 GB SSD PCIe
1x
1x Bluetooth® 5.1 or above; Qualcomm Wi-Fi 6E NFA725A 2x2 AX
1x Headphone & Microphone combo Jack; 2 USB 3.2 Gen 2 (Type A); 2 x USB 4.0 Type-C with PD & DP (Thunderbolt); 1 x RJ45; 1 x HDMI 2.0b; Kensington lock
1x 1080P FHD IR/RGB Hybrid 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=135.0.7049.96)
Stacktrace:
	GetHandleVerifier [0x00007FF6BFBF5355+78597]
	GetHandleVerifier [0x00007FF6BFBF53B0+78688]
	(No symbol) [0x00007FF6BF9A91AA]
	(No symbol) [0x00007FF6BF981B63]
	(No symbol) [0x00007FF6BFA2EB6E]
	(No symbol) [0x00007FF6BFA4EBB2]
	(No symbol) [0x00007FF6BFA26EC3]
	(No symbol) [0x00007FF6BF9F03F8]
	(No symbol) [0x00007FF6BF9F1163]
	GetHandleVerifier [0x00007FF6BFE9EF0D+2870973]
	GetHandleVerifier [0x00007FF6BFE996B8+2848360]
	GetHandleVerifier [0x00007FF6BFEB6993+2967875]
	GetHandleVerifier [0x00007FF6BFC1019A+188746]
	GetHandleVerifier [0x00007FF6BFC1847F+222255]
	GetHandleVerifier [0x00007FF6BFBFD2D4+111236]
	GetHandleVerifier [0x00007FF6BFBFD482+111666]
	GetHandleVerifier [0x00007FF6BFBE35A9+5465]
	BaseThreadInitThunk [0x00007FFDB0017374+20]
	RtlUserThreadStart [0x00007FFDB0FBCC91+33]


In [189]:
import re
import pandas as pd

# Dicionário para mapeamento de nomes (inglês -> português)
mapeamento = {
    "Processor": "Processador",
    "Memory": "Memória",
    "Operating System": "Sistema Operacional",
    "Hard Drive": "Hard Drive",
    "Wired Network": "Rede cabeada",
    "Wireless Network": "Rede sem fio",
    "Ports": "Portas",
    "Camera": "Câmera",
    "Graphics": "Gráficos",
    "Monitor": "Monitores",
    "Form Factor": "Form Factor",
    "Included Warranty": "Garantia inclusa",
    "End of Service": "Fim de serviço"
}

# Função para associar descrição com categorias (considerando inglês e português)
def associar_categoria(descricao):
    for termo_eng, termo_pt in mapeamento.items():
        # Usa igualdade para verificar correspondências exatas
        if descricao.strip().lower() == termo_eng.lower() or descricao.strip().lower() == termo_pt.lower():
            return termo_pt  # Retorna o termo em português como padrão
    return None  # Caso não encontre uma correspondência

# Função para processar o arquivo de texto
def processar_arquivo(arquivo):
    with open(arquivo, "r", encoding="utf-8") as f:
        conteudo = f.read()
    
    # Dividir o conteúdo em blocos (máquinas)
    blocos = re.split(r"------------\s*", conteudo)
    
    dados_extraidos = []
    blocos_nao_processados = []

    for bloco in blocos:
        if "Serial Number:" in bloco:
            maquina = {}
            
            # Extrair o número de série
            serial = re.search(r"Serial Number:\s*(\S+)", bloco)
            if serial:
                maquina["Número de Série"] = serial.group(1)
            
            # Extrair informações que estão abaixo das categorias
            linhas = bloco.split("\n")
            encontrou_categoria = False

            for i in range(len(linhas)):
                linha = linhas[i].strip()
                categoria = associar_categoria(linha)
                if categoria and i + 1 < len(linhas):  # Se categoria identificada e existir linha abaixo
                    maquina[categoria] = linhas[i + 1].strip()
                    encontrou_categoria = True

            if encontrou_categoria:
                dados_extraidos.append(maquina)
            else:
                blocos_nao_processados.append(bloco)

    return dados_extraidos, blocos_nao_processados

# Função para salvar blocos não processados em um novo arquivo
def salvar_blocos_nao_processados(blocos, nome_arquivo):
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        for bloco in blocos:
            f.write(bloco + "\n------------\n")
    print(f"Blocos não processados salvos como {nome_arquivo}")

# Converter os dados em uma planilha
def salvar_para_planilha(dados, nome_arquivo):
    df = pd.DataFrame(dados)
    df.to_excel(nome_arquivo, index=False)
    print(f"Planilha salva como {nome_arquivo}")

# Uso do código
arquivo_txt = "minha_lista_de_maquinas.txt"  # Substitua pelo nome do seu arquivo
dados, blocos_nao_processados = processar_arquivo(arquivo_txt)
salvar_para_planilha(dados, "maquinas.xlsx")
salvar_blocos_nao_processados(blocos_nao_processados, "blocos_nao_processados.txt")


Planilha salva como maquinas.xlsx
Blocos não processados salvos como blocos_nao_processados.txt
